# OAI-PMH und EDM Demo mit der Deutschen Digitalen Bibliothek (DDB)

Die [Deutsche Digitale Bibliothek](https://deutsche-digitale-bibliothek.de/) bietet die Möglichkeit eine große Menge an Kulturdaten frei zu [nutzen](https://pro.deutsche-digitale-bibliothek.de/daten-nutzen). Dafür wird unter Anderem eine [OAI-PMH Schnittstelle](https://pro.deutsche-digitale-bibliothek.de/daten-nutzen/schnittstellen) angeboten. Diese Schnittstelle nutzen wir für ein kleines Beispiel, um sowohl OAI-PMH, als auch das Europeana Data Model (EDM) vorzustellen.

Wir nutzen die [Sickl](https://sickle.readthedocs.io/) Bibliothek, welche uns ein wenig der OAI-PMH-Kommunikation versteckt. Zusätzlich habe ich in der Datei [oai_lib.py](/edit/oai_lib.py) einige weitere Helfer abgelegt.
Der Helfer `get_sickle` gibt uns eine konfigurierte Instanzt der `Sickle` Klasse zurück, welche mit der OAI-PMH-Schnittstelle der DDB verbunden ist.

> Da die späteren Zellen von vorhergehenden Zellen abhängen, sind Zellen, welche unabhängig von anderen Zellen ausgeführt werden können mit `# Einstiegspunkt` gekennzeichnet.

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle
sickle = get_sickle("https://oai.deutsche-digitale-bibliothek.de/", "edm")

## Das Verb ListMetadataFormats

Damit bekommen wir eine Liste der von der DDB angebotenen Metadaten-Formate.
Wir können uns einmal die ganze Antwort der OAI-PMH-Schnittstelle anzeigen lassen … oder auch mit Hilfe der Sickl-Bibliothek durch die einzelnen Einträge iterieren.

Im Angebot sind:
- `oai_dc`: minimales Dublin Core
- `edm`: das Europeana Data Model (EDM), *daran sind wir interessiert*
- `ddb`: das vollständige Austauschformat der DDB, bassierend unter Anderem aud EDM

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle, display_oai
sickle = get_sickle("https://oai.deutsche-digitale-bibliothek.de/", "edm")

metadata_formats = sickle.ListMetadataFormats()

display_oai(metadata_formats)

In [ ]:
# Einstiegspunkt
from oai_lib import get_sickle, display_xmls
sickle = get_sickle("https://oai.deutsche-digitale-bibliothek.de/", "edm")

metadata_formats = sickle.ListMetadataFormats()

for metadata_format in metadata_formats:
    display_xmls(metadata_format)

## Das Verb ListSets

Die DDB bietet Sets mit speziellen Bezeichnungen an:

| Benennung des Datensets | Kommentar |
|-|-|
| `<dataset_id>` | ID des Datensets, die beim Ingest der Daten gebildet wird. |
| `<dataset_id>:<provider_id>` | Provider-ID. Ein Datenset kann Daten mehrere Datenpartner beinhalten. |
| `<dataset_id>:<type_fct>` | Medientyp. Ein Datenpartner möchte z.B. nur die Videos aus einem Datenset liefern. (Eine Liste der Medientypen findet sich im [Wiki der DDB](https://wiki.deutsche-digitale-bibliothek.de/pages/viewpage.action?pageId=27627109#OAI/PMHSchnittstelle-OAI-Datensets)) |
| `<dataset_id>:<license>` | Lizenz. Ein Datenpartner möchte nur die Objekte mit der Lizenz „RV-FZ“ innerhalb eines Datensets liefern. (Eine Liste der Lizenzen findet sich im [Repository der DDB](https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-mapping-new/browse/opt/transformationdir/conf/concordance/common/license_group.xml)) |

(Siehe: [OAI-PMH Schnittstelle](https://pro.deutsche-digitale-bibliothek.de/daten-nutzen/schnittstellen))

Wir möchten Bilder: `mediatype_002` die in der Public Domain: `rights_001` sind.

Wir nutzen die Bezeichnungen der Sets in der DDB und filtern alle Sets heraus, welche ebenfalls Teilsets mit den Bezeichnungen `mediatype_002` und `rights_001` nach dem Doppelpunkt haben.

In [ ]:
# Einstiegspunkt
from collections import defaultdict
from oai_lib import get_sickle
sickle = get_sickle("https://oai.deutsche-digitale-bibliothek.de/", "edm")

oai_sets = sickle.ListSets()

oai_ddb_sets = defaultdict(list)
for oai_set in oai_sets:
    if oai_set.setSpec.find(":") > 0:
        oai_ddb_sets[oai_set.setSpec.split(":")[0]].append(oai_set.setSpec.split(":")[1])

# Hier kommt der Filter
pd_image_sets = list([oai_set for oai_set, set_types in oai_ddb_sets.items() if "mediatype_002" in set_types and "rights_001" in set_types])

print(f"Es sind {len(pd_image_sets)} Sets übrig geblieben, das sind die Identifier: {pd_image_sets}")

## Das Verb ListIdentifiers

Nun iterieren wir über die erhaltenen Sets in `pd_image_sets`, von welchen es jeweils `mediatype_002` (Bild) und `rights_001` (Public Domain) Varianten geben soll.

Um an die Identifier der Records zu gelangen rufen wir das Verb `ListIdentifiers` mit dem entsprechenden Identifier des Sets mit der Variante `mediatype_002` auf.
Die Liste der Identifier der Records wird nochmal filtert, da wir nur an Records interessiert sind, welche auch in `rights_001` (Public Domain) Sets sind. Für unser Beispiel reicht es das Ergebnis auf 5 Einträge zu beschränken.

Zusätzlich benötigt die Schnittselle die Angabe des Metadaten-Formats (`metadataPrefix`) und wir möchten gelöschte Einträge ignorieren.
Falls ein Set leer ist ignorieren wir dieses und gehen zum nächsten Set weiter (`try: … except NoRecordsMatch: continue`).

Die Ausgabe zeigt die Antwort der Schnittstelle auf den Aufruf mit dem Verb `ListIdentifiers`.

In [ ]:
# Hängt von sickle und pd_image_sets ab

from sickle.oaiexceptions import NoRecordsMatch
from oai_lib import display_oai

# set 10194217484138024qoAN "Alice Salomon Archiv der ASH Berlin. Bibliothek"
# set 11408573972398282WpEZ "MONAliesA Feministische Bibliothek"

found_records = []
limit = 5
for oai_set in pd_image_sets[4:]:
    try:
        identifiers = sickle.ListIdentifiers(metadataPrefix="edm", set=oai_set + ":mediatype_002", ignore_deleted=True)
    except NoRecordsMatch:
        continue
    display_oai(identifiers)
    
    for item in identifiers:
        if oai_set + ":rights_001" in item.setSpecs:
            limit -= 1
            if limit < 0: break
            found_records.append(item)
    if limit < 0: break

## Das Verb GetRecord

Nun haben wir eine Liste von Identifiern von Records in `found_records`.
Wir iterieren über diese Liste und rufen die Schnittstelle mit dem Verb `GetRecord` unter Angabe des Identifiers und des gewünschten Metadaten-Formats (`metadataPrefix`) auf. Die Ausgabe zeigt den zurückgegebenen Record an. Die durch die Schnittstellen übertragenen EDM-Metadaten sind innerhalb des `metadata`-Tags als RDF/XML-Graph.
Wir laden die Metadaten der 5 Records in einen gemeinsamen RDF-Graph groß-`G` und heben uns ein einzelnen Beispiel in klein-`g` auf.

Eventuelle Fehler werden ignoriert.

In [ ]:
# Hängt von sickle und found_records ab
from rdflib import Graph
from oai_lib import display_oai

G = Graph()
for item in found_records:
    try:
        record = sickle.GetRecord(metadataPrefix="edm", identifier=item.identifier)
        display_oai(record)
        G = G + record.metadata
        g = record.metadata
    except Exception:
        continue

## Anzeige der EDM-Metadaten als Graph

In [ ]:
# Hängt ab von g

from oai_lib import display_graph

display_graph(g)

## Anzeige der EDM-Metadaten als Turtle-Datei

In [ ]:
# Hängt ab von g

from oai_lib import display_turtle

display_turtle(g)

## Anzeige der ge-harvesteten Medienwerke

In [ ]:
# Hängt ab von G

from IPython.display import display, Image, Markdown

results = G.query("""
prefix dc: <http://purl.org/dc/elements/1.1/>
prefix edm: <http://www.europeana.eu/schemas/edm/>
select ?image {
    ?image a edm:WebResource ;
        dc:format ?format .
    filter(strStarts(?format, "image/"))
}
""")

items = []
for row in results:
    image_url = row["image"]
    items.append(Image(url=row["image"], width=200))
    items.append(Markdown(f"[^link]({image_url})"))

display(*items)